In [29]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
from time import time
import re
import string
import os
import emoji
from pprint import pprint
import collections
import glob

from sklearn.base import BaseEstimator, TransformerMixin

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 

In [30]:
training_path = "data/raw/2018-Valence-oc-En-train.txt"
dev_path = "data/raw/2018-Valence-oc-En-dev.txt"
test_path = "data/raw/2018-Valence-oc-En-test-gold.txt"

training_outpath = "data/cleaned_en_training.txt"
dev_outpath = "data/cleaned_en_dev.txt"
test_outpath = "data/cleaned_en_test.txt"

In [31]:
training_df = pd.read_table(training_path)
dev_df = pd.read_table(dev_path)
test_df = pd.read_table(test_path)

training_df

C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


FileNotFoundError: [Errno 2] File b'data/raw/2018-Valence-oc-En-train.txt' does not exist: b'data/raw/2018-Valence-oc-En-train.txt'

In [42]:
def adapt_valence_scores(df, column_name): 
    valence_list = list()
    for index, row in df.iterrows():
        valence = row[column_name]
        valence = valence.replace(valence, valence[:2].replace(":", ""))
        valence_list.append(valence)
    return valence_list
        
#training_valence = adapt_valence_scores(training_df, "Intensity Class")
#dev_valence = adapt_valence_scores(dev_df)
#test_valence = adapt_valence_scores(test_df)

In [7]:
class CleanText(BaseEstimator, TransformerMixin):
    """
    From https://towardsdatascience.com/sentiment-analysis-with-text-mining-13dd2b33de27
    """
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords)
        return clean_X

In [8]:
ct = CleanText()
training_clean = ct.fit_transform(training_df.Tweet)
dev_clean = ct.fit_transform(dev_df.Tweet)
test_clean = ct.fit_transform(test_df.Tweet)

In [9]:
training_cleaned = pd.DataFrame(training_clean)
training_cleaned['Valence score']= training_valence

dev_cleaned = pd.DataFrame(dev_clean)
dev_cleaned['Valence score']= dev_valence

test_cleaned = pd.DataFrame(test_clean)
test_cleaned['Valence score']= test_valence
test_cleaned = test_cleaned.drop([271]) #Drop empty line from test set

In [10]:
training_cleaned.to_csv(training_outpath, sep="\t")
dev_cleaned.to_csv(dev_outpath, sep="\t")
test_cleaned.to_csv(test_outpath, sep="\t")

In [41]:
#glob.glob('data/V_reg_raw/*')
df = pd.read_table('data/V_reg_raw\\2018-Valence-reg-En-dev.txt')
df
# oc = Intensity Class
# reg = Intensity Score

C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


,ID,Tweet,Affect Dimension,Intensity Score
0,2018-En-02354,So @Ryanair site crashes everytime I try to book - how do they help? Tell me there's nothing wrong &amp; hang up #furious #helpless @SimonCalder,valence,0.141
1,2018-En-00124,"Theme of week: Ask the Lord for strength &amp; perspective to persevere in #integrity and effort, despite being #disheartened &amp; disappointed.",valence,0.317
2,2018-En-02603,"@F1 Why announcing so late, it will be hard to make it from Manchester and organising a day off. #sad",valence,0.145
3,2018-En-00450,The greatest happiness is seeing someone you like stay happy - Daidouji Tomoyo [Cardcaptor Sakura ],valence,0.813
4,2018-En-00502,"omg so grateful to have an education but ive been back at school for 2 (two) days and my back hurts, im exhausted and breaking out already 😍",valence,0.672
5,2018-En-02500,"Because of your @smile, you @make the #life more #beautiful:::::::::::::::::::::::::::::::::::::::::",valence,0.845
6,2018-En-02737,@mashable For some reason this has filled me with delight. See Auntie laugh.,valence,0.774
7,2018-En-01354,@LoveMyFFAJacket FaceTime - we can still annoy you 😂,valence,0.677
8,2018-En-03750,and i shouldve cut them off the moment i started hurting myself over them :o,valence,0.283
9,2018-En-01292,@VescioDiana You forgot #laughter as well ❤️❤️❤️,valence,0.700


In [62]:
folder_path = 'data/V_oc_raw/'
out_folder_path = 

#if __name__ == '__main__':
raw_filepaths = glob.glob(folder_path + '*')
# Loop over raw files
for filepath in raw_filepaths:
    basename = os.path.basename(filepath)
    # If data is English
    basename_ = basename.split('-')
    if len(basename_) == 5: # remove later, just for testing on test.txt
        continue
    # If data is English
    if basename_[3] == 'En':
        print(basename_)
        df = pd.read_table(filepath)
        
        # Get valence score
        if basename_[2] == 'oc':
            val_col = "Intensity Class"
        elif basename_[2] == 'reg':
            val_col = "Intensity Score"
        valence_list = adapt_valence_scores(df, val_col)
        
        ct = CleanText()
        clean_ = ct.fit_transform(df.Tweet)
        
        # Fill empty cells with '[no_text]'
        empty_clean = clean_ == ''
        print(f'{clean_[empty_clean].count()} records have no words left after text cleaning in {basename}')
        clean_.loc[empty_clean] = '[no_text]'
        
        # Add valence score to list
        df_cleaned = pd.DataFrame(clean_)
        df_cleaned['Valence score'] = valence_list
        
        # Drop empty rows
        df_cleaned = df_cleaned[df_cleaned['Tweet'] != '[no_text]']
        df_cleaned.reset_index(drop=True, inplace=True)
        
        # Write to csv
        outpath = 
        #  "data/cleaned_en_training.txt"
        df_cleaned.to_csv(outpath, sep="\t")

    # If data is Arabic
    elif basename_[3] == 'Ar':
        continue

df_cleaned

['2018', 'Valence', 'oc', 'En', 'test', 'gold.txt']


C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  if sys.path[0] == '':


1 records have no words left after text cleaning in 2018-Valence-oc-En-test-gold.txt


,Tweet,Valence score
0,gm tuesday,0
1,lot time tweeting ironic,0
2,graduated yesterday already family members asking job got nightmare,0
3,seriously sitting five minutes watching awe never gets less amazing,1
4,whether glass half empty half full grateful even glass something optimism,2
5,ever fall asleep set alarm lord wakes time anyways 🙌🏻 wonthedoit,1
6,live chicken running around cfa parking lot no one knows came got ironic,0
7,memo going run potus would appreciate help opposition research sarcasm,0
8,tommy phenomenal smashed ajjdin mixes hilarious awesome 😜😂😂👍🏻,3
9,bright early session morning work fullbody ldnmladies surprised actually helped reduce doms,2


In [73]:
df = df_cleaned[270:275]
df = df[df['Tweet'] != '[no_text]']
df.reset_index(drop=True, inplace=True)
df

,Tweet,Valence score
0,un fu ing real glad live uk right heart aches british cousins,-2
1,defendant greets bench cheery hello late arriving court collecting methadone script swansmags,0
2,psa read laying sort forget boiling sun sweat poring face,0
3,weird look creative work not know looks good not 🤷🏻‍♀️ even make sense design eclipseday,1


In [59]:
len(df_cleaned['Tweet'][271])

0